In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib

# -----------------------------
# Load model artifact
# -----------------------------
artifact = joblib.load("resource_optimization_model.joblib")

model = artifact["model"]
le = artifact["area_encoder"]
features = artifact["features"]
threshold = artifact["threshold"]

app = FastAPI(title="Resource Optimization API")


# -----------------------------
# Input schema
# -----------------------------
class PredictionRequest(BaseModel):
    Area_ID: str
    Electricity_kWh: float
    Water_Liters: float
    Temp: float
    Population: int
    DayOfWeek: int
    Month: int
    Weekend: int
    TimeSlot: str  # Morning / Afternoon / Evening


# -----------------------------
# Feature engineering
# -----------------------------
def preprocess(data: PredictionRequest):
    df = pd.DataFrame([data.dict()])

    # Encode Area_ID
    df["Area_ID"] = le.transform(df["Area_ID"])

    # Per-capita features
    df["Elec_per_capita"] = df["Electricity_kWh"] / df["Population"]
    df["Water_per_capita"] = df["Water_Liters"] / df["Population"]

    # Lag & rolling (dummy values for inference)
    # In real deployment, backend should store historical values
    df["Elec_lag_1"] = df["Electricity_kWh"]
    df["Water_lag_1"] = df["Water_Liters"]
    df["Elec_roll_7"] = df["Electricity_kWh"]
    df["Water_roll_7"] = df["Water_Liters"]

    # TimeSlot one-hot encoding
    df["TimeSlot_Morning"] = (df["TimeSlot"] == "Morning").astype(int)
    df["TimeSlot_Afternoon"] = (df["TimeSlot"] == "Afternoon").astype(int)
    df["TimeSlot_Evening"] = (df["TimeSlot"] == "Evening").astype(int)

    df.drop(columns=["TimeSlot"], inplace=True)

    return df[features]


# -----------------------------
# Prediction endpoint
# -----------------------------
@app.post("/predict")
def predict(request: PredictionRequest):
    X = preprocess(request)

    prob = model.predict_proba(X)[:, 1][0]
    optimized = int(prob >= threshold)

    return {
        "optimized": optimized,
        "probability": round(float(prob), 4)
    }

In [ ]:
uvicorn app:app --host 0.0.0.0 --port 8000 # For running